In [ ]:
"""
screen_descriptions: 
a 21x79x80 tensor of utf-8 encodings of textual descriptions of each cell present in the map. NetHack provides these 
textual descriptions (which can be accessed by the user by using the describe action on a specific tile).

pixel:
a representation of the current screen in image form, where each cell is represented by a 16x16x3 image, meaning the 
entire observation is so 336x1264x3 (with 3 channels for RGB).

chars:
a 21x79 matrix of the characters representing the map.

blstats:
a representation of the status line at the bottom of the screen, containing information about the player character's 
position, health, attributes and other statuses. It comes in the form of a dimension 25 vector.

message:
the utf-8 encoding of the on-screen message displayed at the top of the screen. It's a 256-dimensional vector.

inv_strs:
a  55x80 matrix containing utf-8 encodings of textual descriptions of objects present in the current inventory view.
"""

In [1]:
# Per stampare gli id di tutte le mosse/azioni possibili in minihack
from nle.env.base import FULL_ACTIONS
from nle import _pynethack, nethack
MH_FULL_ACTIONS = list(FULL_ACTIONS)
MH_FULL_ACTIONS.remove(nethack.MiscDirection.UP)
print(MH_FULL_ACTIONS)

[<CompassDirection.N: 107>, <CompassDirection.E: 108>, <CompassDirection.S: 106>, <CompassDirection.W: 104>, <CompassDirection.NE: 117>, <CompassDirection.SE: 110>, <CompassDirection.SW: 98>, <CompassDirection.NW: 121>, <CompassDirectionLonger.N: 75>, <CompassDirectionLonger.E: 76>, <CompassDirectionLonger.S: 74>, <CompassDirectionLonger.W: 72>, <CompassDirectionLonger.NE: 85>, <CompassDirectionLonger.SE: 78>, <CompassDirectionLonger.SW: 66>, <CompassDirectionLonger.NW: 89>, <MiscDirection.DOWN: 62>, <MiscDirection.WAIT: 46>, <MiscAction.MORE: 13>, <Command.ADJUST: 225>, <Command.APPLY: 97>, <Command.ATTRIBUTES: 24>, <Command.CALL: 67>, <Command.CAST: 90>, <Command.CHAT: 227>, <Command.CLOSE: 99>, <Command.DIP: 228>, <Command.DROP: 100>, <Command.DROPTYPE: 68>, <Command.EAT: 101>, <Command.ENGRAVE: 69>, <Command.ENHANCE: 229>, <Command.ESC: 27>, <Command.FIGHT: 70>, <Command.FIRE: 102>, <Command.FORCE: 230>, <Command.INVENTORY: 105>, <Command.INVENTTYPE: 73>, <Command.INVOKE: 233>, <Co

In [2]:
len(MH_FULL_ACTIONS)

85

In [3]:
# TUTTE LE AZIONI POSSIBILI LISTATE
"""
[0<CompassDirection.N: 107>, 
 1<CompassDirection.E: 108>, 
 2<CompassDirection.S: 106>, 
 3<CompassDirection.W: 104>, 
 4<CompassDirection.NE: 117>, 
 5<CompassDirection.SE: 110>, 
 6<CompassDirection.SW: 98>, 
 7<CompassDirection.NW: 121>, 
 8<CompassDirectionLonger.N: 75>, 
 9<CompassDirectionLonger.E: 76>, 
 10<CompassDirectionLonger.S: 74>, 
 11<CompassDirectionLonger.W: 72>, 
 12<CompassDirectionLonger.NE: 85>, 
 13<CompassDirectionLonger.SE: 78>, 
 14<CompassDirectionLonger.SW: 66>, 
 15<CompassDirectionLonger.NW: 89>, 
 16<MiscDirection.DOWN: 62>, 
 17<MiscDirection.WAIT: 46>, 
 18<MiscAction.MORE: 13>, 
 19<Command.ADJUST: 225>, 
 20<Command.APPLY: 97>, 
 21<Command.ATTRIBUTES: 24>, 
 22<Command.CALL: 67>, 
 23<Command.CAST: 90>, 
 24<Command.CHAT: 227>, 
 25<Command.CLOSE: 99>, 
 26<Command.DIP: 228>, 
 27<Command.DROP: 100>, 
 28<Command.DROPTYPE: 68>, 
 29<Command.EAT: 101>, 
 30<Command.ENGRAVE: 69>, 
 31<Command.ENHANCE: 229>, 
 32<Command.ESC: 27>, 
 33<Command.FIGHT: 70>, 
 34<Command.FIRE: 102>, 
 35<Command.FORCE: 230>, 
 36<Command.INVENTORY: 105>, 
 37<Command.INVENTTYPE: 73>, 
 38<Command.INVOKE: 233>, 
 39<Command.JUMP: 234>, 
 40<Command.KICK: 4>, 
 41<Command.LOOK: 58>, 
 42<Command.LOOT: 236>, 
 43<Command.MONSTER: 237>, 
 44<Command.MOVE: 109>, 
 45<Command.MOVEFAR: 77>, 
 46<Command.OFFER: 239>, 
 47<Command.OPEN: 111>, 
 48<Command.PAY: 112>, 
 49<Command.PICKUP: 44>, 
 50<Command.PRAY: 240>, 
 51<Command.PUTON: 80>, 
 52<Command.QUAFF: 113>, 
 53<Command.QUIVER: 81>, 
 54<Command.READ: 114>, 
 55<Command.REMOVE: 82>, 
 56<Command.RIDE: 210>, 
 57<Command.RUB: 242>, 
 58<Command.RUSH: 103>, 
 59<Command.RUSH2: 71>, 
 60<Command.SEARCH: 115>, 
 61<Command.SEEARMOR: 91>, 
 62<Command.SEERINGS: 61>, 
 63<Command.SEETOOLS: 40>, 
 64<Command.SEETRAP: 94>, 
 65<Command.SEEWEAPON: 41>, 
 66<Command.SHELL: 33>, 
 67<Command.SIT: 243>, 
 68<Command.SWAP: 120>, 
 69<Command.TAKEOFF: 84>, 
 70<Command.TAKEOFFALL: 65>, 
 71<Command.THROW: 116>, 
 72<Command.TIP: 212>, 
 73<Command.TURN: 244>, 
 74<Command.TWOWEAPON: 88>, 
 75<Command.UNTRAP: 245>, 
 76<Command.VERSIONSHORT: 118>, 
 77<Command.WEAR: 87>, 
 78<Command.WIELD: 119>, 
 79<Command.WIPE: 247>, 
 80<Command.ZAP: 122>, 
 81<TextCharacters.PLUS: 43>, 
 82<TextCharacters.QUOTE: 34>, 
 83<TextCharacters.DOLLAR: 36>, 
 84<TextCharacters.SPACE: 32>]
 """

'\n[0<CompassDirection.N: 107>, \n 1<CompassDirection.E: 108>, \n 2<CompassDirection.S: 106>, \n 3<CompassDirection.W: 104>, \n 4<CompassDirection.NE: 117>, \n 5<CompassDirection.SE: 110>, \n 6<CompassDirection.SW: 98>, \n 7<CompassDirection.NW: 121>, \n 8<CompassDirectionLonger.N: 75>, \n 9<CompassDirectionLonger.E: 76>, \n 10<CompassDirectionLonger.S: 74>, \n 11<CompassDirectionLonger.W: 72>, \n 12<CompassDirectionLonger.NE: 85>, \n 13<CompassDirectionLonger.SE: 78>, \n 14<CompassDirectionLonger.SW: 66>, \n 15<CompassDirectionLonger.NW: 89>, \n 16<MiscDirection.DOWN: 62>, \n 17<MiscDirection.WAIT: 46>, \n 18<MiscAction.MORE: 13>, \n 19<Command.ADJUST: 225>, \n 20<Command.APPLY: 97>, \n 21<Command.ATTRIBUTES: 24>, \n 22<Command.CALL: 67>, \n 23<Command.CAST: 90>, \n 24<Command.CHAT: 227>, \n 25<Command.CLOSE: 99>, \n 26<Command.DIP: 228>, \n 27<Command.DROP: 100>, \n 28<Command.DROPTYPE: 68>, \n 29<Command.EAT: 101>, \n 30<Command.ENGRAVE: 69>, \n 31<Command.ENHANCE: 229>, \n 32<Comma

In [4]:
# Queste azioni sono quelle che poi vengono passate e env.step(id_action)
# Eccone alcune delle più importanti:
# DIREZIONI
print(MH_FULL_ACTIONS[0])
print(MH_FULL_ACTIONS[1])
print(MH_FULL_ACTIONS[2])
print(MH_FULL_ACTIONS[3])
print(MH_FULL_ACTIONS[44])

CompassDirection.N
CompassDirection.E
CompassDirection.S
CompassDirection.W
Command.MOVE


In [ ]:
import gym
import time
import matplotlib.pyplot as plt
from pyswip import Prolog
import IPython.display as display
from minihack import LevelGenerator
from minihack import RewardManager
from minihack.reward_manager import SequentialRewardManager
import random
from nle import nethack
from nle.nethack import Command
import numpy as np
STATS = ['x', 'y', 'strength_percentage', 'strength', 'dexterity', 'constitution', 'intelligence',
'wisdom', 'charisma', 'score', 'hitpoints (health)', 'max_hitpoints', 'depth', 'gold',
'energy', 'max_energy', 'armor_class', 'monster_level', 'experience_level',
'experience_points', 'time', 'hunger_state', 'carrying_capacity',
'dungeon_number', 'level_number']
def print_stats(blstats):
stat_vals = list(blstats)
i = 0
for s in STATS:
print(s + ': ' + str(stat_vals[i]))
i += 1